In [76]:
import pandas as pd

In [77]:
dataset = pd.read_csv('GPUs.csv', skiprows=1, sep=';')

In [78]:
dataset.head(100)

,Nazwa,Koszt,Wydajnosc,TDP,Ocena
0,Nvidia RTX 3060-Ti,2109,132.0,220,99
1,Nvidia RTX 3070,2383,154.0,220,99
2,Nvidia RTX 3050,1408,71.7,130,96
3,Nvidia GTX 1660S,1043,70.5,125,94
4,Nvidia RTX 3080,4266,206.0,320,91
5,Nvidia RTX 4090,9699,422.0,450,90
6,Nvidia RTX 2060,1493,88.6,175,87
7,Nvidia RTX 4080,7049,331.0,320,66
8,Nvidia RTX 3090-Ti,6499,266.0,450,60
9,Nvidia RTX 3090,5767,233.0,350,71


In [79]:
dataset.describe()

,Koszt,Wydajnosc,TDP,Ocena
count,21.000000,21.000000,21.000000,21.000000
mean,4087.809524,167.376190,271.000000,70.000000
std,2451.496352,90.338192,89.145948,17.997222
min,1043.000000,70.500000,125.000000,50.000000
25%,2109.000000,96.200000,220.000000,57.000000
50%,3599.000000,147.000000,250.000000,60.000000
75%,6162.000000,206.000000,320.000000,90.000000
max,9699.000000,422.000000,450.000000,99.000000


# Promethe

In [80]:
# Preference funcions

def pref_v_shape(d, q, p):
    if d > p:
        return 1
    elif d <= q:
        return 0
    else:
        return (d - q) / (p - q)

In [81]:
def comprehensive_preference_index(item1, item2, pref_info):
    val = 0.0
    for i in range(len(pref_info)):
        g = pref_info.iloc[i].name
        w = pref_info.iloc[i]["w"]
        q = pref_info.iloc[i]["q"]
        p = pref_info.iloc[i]["p"]
        a = item1[g]
        b = item2[g]
        d = a - b if pref_info.iloc[i]["cost_or_gain"] == "g" else b - a
        pref = pref_info.iloc[i]["pref_func"](d, q, p)
        val += pref * w
    
    val /= pref_info["w"].sum()

    return val 

In [82]:
pref_info = pd.DataFrame({
    "g": ["Koszt", "Wydajnosc", "TDP", "Ocena"],
    "q": [100, 20, 10, 5],
    "p": [300, 50, 30, 30],
    "w": [3, 4, 1, 2],
    "cost_or_gain": ["c", "g", "c", "g"],
    "pref_func": [pref_v_shape]*4
})

pref_info = pref_info.set_index("g")

In [83]:
pref_info

,q,p,w,cost_or_gain,pref_func
g,,,,,
Koszt,100,300,3,c,<function pref_v_shape at 0x0000014F88089820>
Wydajnosc,20,50,4,g,<function pref_v_shape at 0x0000014F88089820>
TDP,10,30,1,c,<function pref_v_shape at 0x0000014F88089820>
Ocena,5,30,2,g,<function pref_v_shape at 0x0000014F88089820>


In [84]:
pref_info.iloc[0] 

q                                                         100
p                                                         300
w                                                           3
cost_or_gain                                                c
pref_func       <function pref_v_shape at 0x0000014F88089820>
Name: Koszt, dtype: object

In [85]:
pref_info.loc["Koszt"]

q                                                         100
p                                                         300
w                                                           3
cost_or_gain                                                c
pref_func       <function pref_v_shape at 0x0000014F88089820>
Name: Koszt, dtype: object

In [86]:
comprehensive_preference_index(dataset.iloc[0], dataset.iloc[1], pref_info)

0.261

In [87]:
comp_table = []

for i in range(len(dataset)):
    item1 = dataset.iloc[i]
    row = []
    for j in range(len(dataset)):
        item2 = dataset.iloc[j]
        result = comprehensive_preference_index(item1, item2, pref_info)
        row.append(result)
    comp_table.append(row)

comp_df = pd.DataFrame(comp_table)
flow_plus = comp_df.sum(axis=1)
flow_minus = comp_df.sum(axis=0)
flow = flow_plus - flow_minus
comp_df['flow+'] = flow_plus
comp_df['flow-'] = flow_minus
comp_df['flow'] = flow

In [88]:
comp_df.sort_values("flow", ascending=False)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,flow+,flow-,flow
1,0.026667,0.000000,0.400000,0.400000,0.424000,0.432,0.456000,0.600,0.600,0.584000,...,0.793333,0.600000,0.600000,0.600000,0.866667,0.900000,0.600,11.402667,4.854333,6.548333
0,0.000000,0.261000,0.400000,0.400000,0.424000,0.432,0.368000,0.600,0.600,0.584000,...,0.500000,0.448000,0.553333,0.600000,0.600000,0.710667,0.600,10.481000,5.101667,5.379333
3,0.400000,0.400000,0.300000,0.000000,0.400000,0.400,0.416000,0.584,0.600,0.544000,...,0.600000,0.600000,0.600000,0.600000,0.600000,0.600000,0.600,10.644000,5.914667,4.729333
2,0.400000,0.400000,0.000000,0.000000,0.400000,0.408,0.132000,0.600,0.600,0.560000,...,0.600000,0.600000,0.526500,0.600000,0.600000,0.600000,0.600,10.026500,6.150667,3.875833
4,0.400000,0.400000,0.400000,0.400000,0.000000,0.400,0.400000,0.460,0.600,0.520000,...,0.600000,0.600000,0.600000,0.600000,0.600000,0.600000,0.525,10.269000,6.594667,3.674333
6,0.400000,0.400000,0.000000,0.000000,0.400000,0.400,0.000000,0.528,0.576,0.488000,...,0.584000,0.600000,0.399000,0.576000,0.600000,0.500000,0.600,9.379000,5.708000,3.671000
5,0.400000,0.400000,0.400000,0.400000,0.400000,0.000,0.400000,0.552,0.600,0.512000,...,0.600000,0.600000,0.600000,0.600000,0.600000,0.600000,0.600,10.640000,7.972000,2.668000
9,0.400000,0.400000,0.400000,0.400000,0.093333,0.400,0.400000,0.300,0.448,0.000000,...,0.456000,0.512000,0.528000,0.348000,0.512000,0.496000,0.764,8.960000,7.790333,1.169667
11,0.200000,0.000000,0.400000,0.400000,0.400000,0.400,0.400000,0.400,0.400,0.400000,...,0.400000,0.448000,0.464000,0.400000,0.748000,0.432000,0.440,7.933333,6.789333,1.144000
7,0.400000,0.400000,0.400000,0.400000,0.400000,0.400,0.400000,0.000,0.508,0.500000,...,0.416000,0.472000,0.488000,0.508000,0.472000,0.456000,0.789,9.081000,8.324000,0.757000


In [89]:
comp_df.sort_values("flow", ascending=False)[['flow+', 'flow-', 'flow']]

,flow+,flow-,flow
1,11.402667,4.854333,6.548333
0,10.481000,5.101667,5.379333
3,10.644000,5.914667,4.729333
2,10.026500,6.150667,3.875833
4,10.269000,6.594667,3.674333
6,9.379000,5.708000,3.671000
5,10.640000,7.972000,2.668000
9,8.960000,7.790333,1.169667
11,7.933333,6.789333,1.144000
7,9.081000,8.324000,0.757000
